In [ ]:
from __future__ import print_function
from visdom import Visdom
import torch.utils.model_zoo as model_zoo
import logging
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import numpy as np
import os
import matplotlib.pyplot as plt
import time
from PIL import Image

In [ ]:
# Visdom line plotter object
class VisdomLinePlotter(object):
    def __init__(self, env_name='main'):
        self.vis = Visdom()
        self.env = env_name
        self.plots = {}
    def plot(self, var_name, split_name, title_name, x, y):
        if var_name not in self.plots:
            self.plots[var_name] = self.vis.line(X=np.array([x,x]), Y=np.array([y,y]), env=self.env, opts=dict(
                legend=[split_name],
                title=title_name,
                xlabel='Epochs',
                ylabel=var_name
            ))
        else:
            self.vis.line(X=np.array([x]), Y=np.array([y]), env=self.env, win=self.plots[var_name], name=split_name, update = 'append')


In [ ]:
# Model definition
class AlexNet(nn.Module):
    def __init__(self, num_classes=3):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),   
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),            #should yield 1,64,16,16,
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),   # # channels,#output size, #stride, #padding
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2), #1,128,8,8
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),   # # channels,#output size, #stride, #padding
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2) #1,256,4,4
        )

        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 4 * 4, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 4 *4 )
        x = self.classifier(x)
        return x

In [ ]:
def trains(model, device, train_loader, optimizer, epoch,celoss):
    model.train()
    log_interval=10;
    correct=0;
    total_loss=0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        #print(target)
        optimizer.zero_grad()
        output = model(data)
        loss = celoss(output, target).sum()
        loss.backward()
        optimizer.step()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        total_loss=loss+total_loss
    train_accuracy=(100. * correct) / len(train_loader.dataset) 
    train_loss=total_loss.item()
    train_loss=train_loss/len(train_loader)
    
    print('Train Epoch:{},Accuracy:{:.4f},Loss:{:.6f},'.format(epoch, 100.* correct / len(train_loader.dataset)
        ,train_loss),end="")
    logging.info('Train Epoch:{},Accuracy:{:.4f},Loss:{:.6f},'.format(epoch, 100.* correct / len(train_loader.dataset)
        ,train_loss))
    return train_accuracy, train_loss

def test(model, device, test_loader,celoss):
    model.eval()
    total_test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss = celoss(output, target)
            #test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1]  
            total_test_loss=test_loss+total_test_loss
            # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
        
    total_test_loss /= len(test_loader)
    test_acc=100. * correct / len(test_loader.dataset)
    print('Test set:Accuracy:{:.4f},loss:{:.6f}'.format(100. * correct / len(test_loader.dataset),
        total_test_loss))
    logging.info('Test set:Accuracy:{:.4f},loss:{:.6f}'.format(100. * correct / len(test_loader.dataset),
        total_test_loss))
    
    return test_acc,total_test_loss


In [ ]:
# Data loader and Data transforms

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batch_size1=2
data_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.7),  
            transforms.RandomVerticalFlip(p=0.7),
            transforms.RandomGrayscale(),
            transforms.RandomRotation(45),
            transforms.ColorJitter(),
            transforms.ToTensor()
        ])
data_transform_test = transforms.Compose([
            transforms.ToTensor()
        ])
train_data = datasets.ImageFolder('/Users/goutham/Development/DATASETS/Patch_based__HE_sampler/train',data_transform)
train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size1,shuffle=True)

test_data = torchvision.datasets.ImageFolder('/Users/goutham/Development/DATASETS/Patch_based__HE_sampler/val',data_transform)
test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size1,shuffle=True)

In [ ]:
# Run data through the model

plotter = VisdomLinePlotter()  # Make an instance of Line plotter object defined above
logging.basicConfig(filename='CNN_visdom_tutorial_classification',level=logging.INFO)
model = AlexNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.5)
epochs=50

celoss = nn.CrossEntropyLoss()
acc_data=np.zeros((2,epochs), dtype=float)
loss_data=np.zeros((2,epochs), dtype=float)

for epoch in range(1, epochs + 1):
    train_acc,train_loss=trains(model, device, train_loader, optimizer, epoch, celoss)  
    plotter.plot('acc', 'train', 'Class Accuracy', epoch, train_acc) # Plot the accuracy training 
    plotter.plot('loss', 'train', 'Loss', epoch, train_loss) # Plot loss testing
    acc_data[0,epoch-1]=train_acc
    loss_data[0,epoch-1]=train_loss
    test_acc,test_loss=test(model,device,test_loader,celoss)
    plotter.plot('acc', 'test', 'Accuracy', epoch, test_acc) # Plot accuracy training
    plotter.plot('loss','test','Loss',epoch,test_loss) # Plot loss testing
    
    acc_data[1,epoch-1]=test_acc
    loss_data[1,epoch-1]=test_loss
    